In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

%matplotlib inline
%run load_data.ipynb

In [2]:
word_count = get_word_counts() #TAKE OUT STORE DATA IN FUNCTIONS IN CASE CHANGES OF DATABASE
stock_prices = get_stock_prices()

word_data = pd.DataFrame(data=word_count, columns=['id', 'word', 'count', 'date'])
stock_data = pd.DataFrame(data=stock_prices, columns=['id', 'openprice', 'closeprice', 'date'])

In [3]:
word_data.head()

id                word  count        date
0   1            birthday      1  2018-03-07
1   2  httpstco8taibf6hls      1  2018-03-07
2   3          production      1  2018-03-07
3   4               tesla      2  2018-03-07
4   5                doom      1  2018-03-07

In [4]:
stock_data.head()

id  openprice  closeprice        date
0   1     253.05      249.24  2017-02-01
1   2     248.34      251.55  2017-02-02
2   3     251.91      251.33  2017-02-03
3   4     251.00      257.77  2017-02-06
4   5     258.19      257.48  2017-02-07

In [5]:
'''
print("Max stock data: {}".format(np.max(stock_data['date'])))
print("Max tweet date: {}".format(np.max(word_data['date'])))
'''

'\nprint("Max stock data: {}".format(np.max(stock_data[\'date\'])))\nprint("Max tweet date: {}".format(np.max(word_data[\'date\'])))\n'

In [6]:
### Accumulate tweets from weekends and holidays to the next open stock market day

In [7]:
'''
last_stock_date = np.max(stock_data['date'])
word_data.drop(labels=word_data.loc[word_data['date'] > last_stock_date], axis=1)
'''
while(pd.unique(word_data['date'].isin(stock_data['date'])).shape[0] > 1):
    word_data.loc[~word_data['date'].isin(stock_data['date']), 'date'] += dt.timedelta(days=1) 


In [8]:
### Getting Percentage stock increase by word count

In [9]:
agg = pd.merge(stock_data, word_data, how='right', on='date', sort=True, suffixes=('_stock', '_word'))

agg.head()

id_stock  openprice  closeprice        date  id_word                word  \
0        14     275.45      277.39  2017-02-21     7885                  no   
1        14     275.45      277.39  2017-02-21     7886                  rt   
2        14     275.45      277.39  2017-02-21     7887                been   
3        14     275.45      277.39  2017-02-21     7888  httpstco6ve3yjostm   
4        14     275.45      277.39  2017-02-21     7889              falcon   

   count  
0      1  
1      2  
2      1  
3      1  
4      1

## Build word_stats table:

In [14]:
word_stats = pd.DataFrame(pd.unique(agg['word']), columns=['word'])
word_stats['count'] = word_stats['word'].apply(lambda x: np.sum(agg.loc[agg['word'] == x, 'count']))

In [15]:
total_words = np.sum(word_stats['count'])
word_stats['prob_occur'] = word_stats['count'].apply(lambda x: x / total_words)

word_stats['prob_inc'] = word_stats['word'].apply(lambda x: np.sum(agg.loc[(agg['word'] == x) & (agg['closeprice'] - agg['openprice'] >= 0), 'count']))
word_stats['prob_inc'] = word_stats.apply(lambda x: x['prob_inc'] / x['count'], axis=1)

word_stats['prob_dec'] = word_stats.apply(lambda x: 1.0 - x['prob_inc'], axis = 1)
word_stats.head()

word_stats

word  count  prob_occur  prob_inc  prob_dec
0                     no    101    0.002633  0.534653  0.465347
1                     rt    320    0.008341  0.496875  0.503125
2                   been     34    0.000886  0.617647  0.382353
3     httpstco6ve3yjostm      1    0.000026  1.000000  0.000000
4                 falcon     97    0.002528  0.453608  0.546392
5                   back     28    0.000730  0.607143  0.392857
6                  solar     22    0.000573  0.681818  0.318182
7               deployed      4    0.000104  1.000000  0.000000
8                  video     24    0.000626  0.416667  0.583333
9                 dragon     33    0.000860  0.484848  0.515152
10                posted      4    0.000104  0.500000  0.500000
11            spacecraft      4    0.000104  0.750000  0.250000
12    httpstcomzkw5udsti      1    0.000026  1.000000  0.000000
13                  came      4    0.000104  1.000000  0.000000
14                  have    132    0.003441  0.575758  0.424242
15                   has     89    0.002320  0.573034  0.426966
16             mattnewby      1    0.000026  1.000000  0.000000
17                 cargo     10    0.000261  0.800000  0.200000
18                  just    130    0.003389  0.476923  0.523077
19    httpstco5frhw3at2b      1    0.000026  1.000000  0.000000
20                landed     11    0.000287  0.545455  0.454545
21                    on    269    0.007012  0.457249  0.542751
22                     a    735    0.019158  0.563265  0.436735
23                   lz1      3    0.000078  0.666667  0.333333
24                spacex    205    0.005343  0.517073  0.482927
25                 right     59    0.001538  0.525424  0.474576
26    httpstcorcv7mvi8rs      1    0.000026  1.000000  0.000000
27              schedule      3    0.000078  0.333333  0.666667
28                  nasa     37    0.000964  0.594595  0.405405
29                    at    167    0.004353  0.502994  0.497006
...                  ...    ...         ...       ...       ...
8892              locked      1    0.000026  1.000000  0.000000
8893             configs      1    0.000026  1.000000  0.000000
8894  httpstcoiju6nhst99      1    0.000026  1.000000  0.000000
8895  httpstco5knllralxh      1    0.000026  1.000000  0.000000
8896                meme      1    0.000026  0.000000  1.000000
8897  httpstcoazh4eorgmc      1    0.000026  0.000000  1.000000
8898  httpstcoedu5ujvvne      1    0.000026  0.000000  1.000000
8899                puro      1    0.000026  0.000000  1.000000
8900              dealer      1    0.000026  0.000000  1.000000
8901             dankest      1    0.000026  0.000000  1.000000
8902               fresh      1    0.000026  0.000000  1.000000
8903     tylerthecreator      1    0.000026  1.000000  0.000000
8904  httpstcoie4ybmw0tw      1    0.000026  1.000000  0.000000
8905  httpstcoulgsz1wlpl      1    0.000026  1.000000  0.000000
8906              titles      1    0.000026  0.000000  1.000000
8907                rare      1    0.000026  0.000000  1.000000
8908  httpstcofuynq0o0bo      1    0.000026  1.000000  0.000000
8909             uncover      1    0.000026  1.000000  0.000000
8910              sneaky      1    0.000026  1.000000  0.000000
8911  httpstco3kttkljwjy      1    0.000026  1.000000  0.000000
8912          overtheair      1    0.000026  1.000000  0.000000
8913  httpstcoscqpeofnpg      1    0.000026  1.000000  0.000000
8914        disturbingly      1    0.000026  1.000000  0.000000
8915        ridiculously      1    0.000026  1.000000  0.000000
8916              budget      1    0.000026  1.000000  0.000000
8917              walked      1    0.000026  1.000000  0.000000
8918             budgets      1    0.000026  1.000000  0.000000
8919      onedimensional      1    0.000026  1.000000  0.000000
8920                 11m      1    0.000026  1.000000  0.000000
8921  httpstcogcrl2un7j7      1    0.000026  1.000000  0.000000

[8922 rows x 5 columns]

In [18]:
# word_stats[['word', 'prob_inc', 'prob_occur']].to_csv('word_stats.csv')

# Build Naive Bayes Estimate:

## p(y|x) = (p(x|y) * p(y)) / p(x)
p(x|y) = probability of stock increase given the word  
p(x|y) = probability of the word is said given stock increase  
p(y) = probability of stock increase  
p(x) = probabaility of word  

In [12]:
prob_stock_inc = agg.loc[agg['closeprice'] - agg['openprice'] >= 0, :].shape[0] / agg.shape[0]
print("Probability of stock increase: {}".format(prob_stock_inc))

agg.loc[stock_data.loc[2, 'date'] == agg['date']

stock_data['inc_pred'] = stock_data.apply(lambda x: word_stats.loc[agg.loc[agg['date'] == x['date'], 'word'], 'prob_inc'].apply(lambda y: prob_stock_inc * y).product(), axis=1)
#stock_data['dec_pred'] = 
#stock_data['increase'] = stock_data.apply(lambda x: True if x['inc_pred'] >= x['dec_pred'] else False)

SyntaxError: invalid syntax (<ipython-input-12-788cbcb618a6>, line 6)